# Transforming data

In [52]:
from pyspark.sql import SparkSession

spark=SparkSession.builder \
        .appName("Superstore Clean and Transform") \
        .getOrCreate()


In [53]:
file_path="/home/daman/Downloads/notebooks/SalesTrac/data/raw/Superstore.csv"
df=spark.read.csv(file_path,header=True,inferSchema=True)
df.show(5)


+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|  Customer Name|  Segment|      Country|           City|     State|Postal Code|Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      42420| South|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset 

In [54]:
df.printSchema()
df.count()

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)



9994

## Clean and Transform

In [55]:
from pyspark.sql.functions import col

for col_name in df.columns:
    new_col=col_name.strip().lower().replace(" ","_")
    df=df.withColumnRenamed(col_name,new_col)


df = df.withColumn("sales", col("sales").cast("double")) \
       .withColumn("quantity", col("quantity").cast("int")) \
       .withColumn("discount", col("discount").cast("double"))    

df.printSchema()  

root
 |-- row_id: integer (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- ship_date: string (nullable = true)
 |-- ship_mode: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- segment: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: integer (nullable = true)
 |-- region: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sub-category: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- sales: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- discount: double (nullable = true)
 |-- profit: double (nullable = true)



In [56]:
from pyspark.sql.functions import to_date
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

df=df.withColumn("order_date",to_date("order_date","MM/dd/yyyy"))
df=df.withColumn("ship_date",to_date("ship_date","MM/dd/yyyy"))

df.select("order_date","ship_date").show(5)

+----------+----------+
|order_date| ship_date|
+----------+----------+
|2016-11-08|2016-11-11|
|2016-11-08|2016-11-11|
|2016-06-12|2016-06-16|
|2015-10-11|2015-10-18|
|2015-10-11|2015-10-18|
+----------+----------+
only showing top 5 rows



In [57]:
from pyspark.sql.functions import year,month

df=df.withColumn("order_year",year("order_date"))
df=df.withColumn("order_month",month("order_date"))
df.select("order_year","order_month").show(5)

+----------+-----------+
|order_year|order_month|
+----------+-----------+
|      2016|         11|
|      2016|         11|
|      2016|          6|
|      2015|         10|
|      2015|         10|
+----------+-----------+
only showing top 5 rows



In [58]:
df.show(5)
df.count()

+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+----------+-----------+
|row_id|      order_id|order_date| ship_date|     ship_mode|customer_id|  customer_name|  segment|      country|           city|     state|postal_code|region|     product_id|       category|sub-category|        product_name|   sales|quantity|discount|  profit|order_year|order_month|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+----------+-----------+
|     1|CA-2016-152156|2016-11-08|2016-11-11|  Second Class|   CG-12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      424

9994

In [59]:
df=df.dropna(subset=["order_id","product_id"])
df=df.dropDuplicates(["order_id","product_id"])
df.count()

9986

## Cleaned Parquet File

In [60]:
df.write.partitionBy("order_year").mode("overwrite")\
    .parquet("/home/daman/Downloads/notebooks/SalesTrac/data/processed")
print("Stored")


Stored


# Incremental Load

In [61]:
existing_df = spark.read.parquet("/home/daman/Downloads/notebooks/SalesTrac/data/processed",header=True, inferSchema=True)
existing_df.count()
existing_df.printSchema()

root
 |-- row_id: integer (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- ship_date: date (nullable = true)
 |-- ship_mode: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- segment: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: integer (nullable = true)
 |-- region: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sub-category: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- sales: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- discount: double (nullable = true)
 |-- profit: double (nullable = true)
 |-- order_month: integer (nullable = true)
 |-- order_year: integer (nullable = true)



## New data

In [62]:
new_df = spark.read.csv("/home/daman/Downloads/notebooks/SalesTrac/data/raw/superstore_2023_2025_1000.csv", header=True, inferSchema=True)
new_df.show(5)
new_df.count()

+------+--------------+----------+----------+------------+-----------+-------------+-----------+-------------+---------+----------+-----------+------+-------------+---------------+------------+--------------+-------+--------+--------+-------+
|Row ID|      Order ID|Order Date| Ship Date|   Ship Mode|Customer ID|Customer Name|    Segment|      Country|     City|     State|Postal Code|Region|   Product ID|       Category|Sub-Category|  Product Name|  Sales|Quantity|Discount| Profit|
+------+--------------+----------+----------+------------+-----------+-------------+-----------+-------------+---------+----------+-----------+------+-------------+---------------+------------+--------------+-------+--------+--------+-------+
|  9995|CA-2025-779294|05/14/2025|05/17/2025| First Class|   MB-23440| Steve Turner|  Corporate|United States|  Houston|     Texas|      77001|  East|OFF-BI-797773|Office Supplies|     Binders| 3-Ring Binder|1956.56|       3|    0.05|1173.94|
|  9996|CA-2025-975558|11/28

1000

## Clean and Transform New data

In [63]:
from pyspark.sql.functions import to_date, year, month

# Rename columns
for col_name in new_df.columns:
    new_df = new_df.withColumnRenamed(col_name, col_name.strip().lower().replace(" ", "_"))

# Fix date format
new_df = new_df.withColumn("order_date", to_date("order_date", "MM/dd/yyyy"))
new_df = new_df.withColumn("ship_date", to_date("ship_date", "MM/dd/yyyy"))

# Add year/month for partitioning
new_df = new_df.withColumn("order_year", year("order_date"))
new_df = new_df.withColumn("order_month", month("order_date"))

new_df.show(5)


+------+--------------+----------+----------+------------+-----------+-------------+-----------+-------------+---------+----------+-----------+------+-------------+---------------+------------+--------------+-------+--------+--------+-------+----------+-----------+
|row_id|      order_id|order_date| ship_date|   ship_mode|customer_id|customer_name|    segment|      country|     city|     state|postal_code|region|   product_id|       category|sub-category|  product_name|  sales|quantity|discount| profit|order_year|order_month|
+------+--------------+----------+----------+------------+-----------+-------------+-----------+-------------+---------+----------+-----------+------+-------------+---------------+------------+--------------+-------+--------+--------+-------+----------+-----------+
|  9995|CA-2025-779294|2025-05-14|2025-05-17| First Class|   MB-23440| Steve Turner|  Corporate|United States|  Houston|     Texas|      77001|  East|OFF-BI-797773|Office Supplies|     Binders| 3-Ring B

## Compare and filter new records

In [64]:
join_cond = [
    new_df["order_id"] == existing_df["order_id"],
    new_df["product_id"] == existing_df["product_id"]
]

# Anti join to get only truly new rows
incremental_df = new_df.join(existing_df, join_cond, "left_anti")

print("New records found:", incremental_df.count())
incremental_df.select("order_id", "product_id", "order_date").show(5)


New records found: 1000
+--------------+-------------+----------+
|      order_id|   product_id|order_date|
+--------------+-------------+----------+
|CA-2025-779294|OFF-BI-797773|2025-05-14|
|CA-2025-975558|FUR-TA-700975|2025-11-28|
|CA-2023-322765|TEC-AC-536540|2023-09-01|
|CA-2025-400102|TEC-PH-385668|2025-09-26|
|CA-2024-828295|OFF-ST-324010|2024-09-10|
+--------------+-------------+----------+
only showing top 5 rows



## Append new data to cleaned parquet

In [65]:
incremental_df.write.partitionBy("order_year").mode("append") \
    .parquet("/home/daman/Downloads/notebooks/SalesTrac/data/processed")

print("Incremental records appended.")


Incremental records appended.
